In [ ]:
from pasteur.kedro.ipython import *
register_kedro()

INFO     Kedro project Pasteur                                                                                           __init__.py:77
INFO     Defined global variable 'context', 'session', 'catalog' and 'pipelines'                                         __init__.py:78
INFO     Registered line magic 'run_viz'                                                                                 __init__.py:84
INFO     Registered line magic 'reload_kedro_mlflow'                                                                     __init__.py:84


In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from pasteur.metadata import Metadata

trn: pd.DataFrame = catalog.load("mimic_tab_admissions.trn.table")
table = catalog.load("mimic_tab_admissions.wrk.idx_table")

In [ ]:
attrs = trn["idx"].get_attributes()

In [ ]:
from copy import copy
from pasteur.synth.hierarchy import rebalance_column

new_attrs = {}
for name, attr in attrs.items():
    cols = {}
    for col_name, col in attr.cols.items():
        cols[col_name] = rebalance_column(
            table[col_name], col, 0
        )

    new_attr = copy(attr)
    new_attr.update_cols(cols)
    new_attrs[name] = new_attr
new_attrs


{
    'birth_year': Attr[na=0,ukn=0]{},
    'dod': Attr[na=1,ukn=0]{'dod_year': <pasteur.synth.hierarchy.RebalancedColumn object at 0x7f143013ccd0>, 'dod_week': <pasteur.synth.hierarchy.RebalancedColumn object at 0x7f143013cca0>, 'dod_day': <pasteur.synth.hierarchy.RebalancedColumn object at 0x7f143013cc40>},
    'admittime': Attr[na=0,ukn=0]{'admittime_year': <pasteur.synth.hierarchy.RebalancedColumn object at 0x7f143013c1c0>, 'admittime_week': <pasteur.synth.hierarchy.RebalancedColumn object at 0x7f143013da20>, 'admittime_day': <pasteur.synth.hierarchy.RebalancedColumn object at 0x7f143013d9c0>, 'admittime_time': <pasteur.synth.hierarchy.RebalancedColumn object at 0x7f143013c040>},
    'dischtime': Attr[na=0,ukn=0]{'dischtime_day': <pasteur.synth.hierarchy.RebalancedColumn object at 0x7f143013d1e0>, 'dischtime_time': <pasteur.synth.hierarchy.RebalancedColumn object at 0x7f143013d240>},
    'deathtime': Attr[na=1,ukn=0]{'deathtime_day': <pasteur.synth.hierarchy.RebalancedColumn objec

In [ ]:
ZERO_FILL = 1e-24
grouping = new_attrs["dod"]["dod_year"].grouping
column = table["dod_year"]

mapped = grouping[:, column]
counts = []
domains = np.max(grouping, axis=1) + 1

for i in range(len(mapped)):
    counts.append(np.bincount(mapped[i, :]))

entropies = []
for count in counts:
    p = count / count.max() + ZERO_FILL
    entropies.append(-np.sum(p * np.log2(p)))

entropies = np.array(entropies)
entropies_orig = entropies

entropies, np.log2(domains)


(
    array([0.57051281, 0.57051281, 0.57051281, 0.57051281, 0.57051281,
       0.5704901 , 0.57033927, 0.56983351, 0.56904794, 0.56808503,
       0.56704387, 0.56529822, 0.56361042, 0.56107146, 0.55832771,
       0.55486176, 0.54989038, 0.54361104, 0.53674918, 0.52901496,
       0.52013597, 0.51093399, 0.5012749 , 0.49003703, 0.47908775,
       0.46561534, 0.44977822, 0.43117611, 0.40784666, 0.37856016,
       0.33974806, 0.2700249 ]),
    array([5.043, 5.   , 4.953, 4.906, 4.86 , 4.81 , 4.754, 4.7  , 4.645,
       4.586, 4.523, 4.46 , 4.39 , 4.32 , 4.246, 4.17 , 4.086, 4.   ,
       3.906, 3.807, 3.701, 3.586, 3.459, 3.322, 3.17 , 3.   , 2.807,
       2.586, 2.322, 2.   , 1.585, 1.   ], dtype=float16)
)


In [ ]:
entropies / np.log2(domains)


array([0.11313035, 0.11410256, 0.1151824 , 0.11628287, 0.11740457,
       0.1186397 , 0.11997276, 0.12126133, 0.12251999, 0.12387544,
       0.12535685, 0.12672184, 0.12836678, 0.12986826, 0.13149208,
       0.13300057, 0.1345812 , 0.13590276, 0.13740779, 0.13897161,
       0.14053278, 0.14248268, 0.14491968, 0.14750086, 0.15113551,
       0.15520511, 0.16025501, 0.1667388 , 0.17562447, 0.18928008,
       0.21435737, 0.2700249 ])


In [ ]:
u = 1.5
x = (1 + u/np.log2(domains))*entropies
x / x.max()


array([0.97835785, 0.98056799, 0.98277814, 0.98498829, 0.98719843,
       0.99010588, 0.99205359, 0.99411721, 0.99568601, 0.99693549,
       0.99876953, 0.99861473, 1.        , 0.99984233, 0.99927879,
       0.99737457, 0.99411908, 0.9883828 , 0.98214475, 0.97550706,
       0.96652244, 0.95800035, 0.95024649, 0.94033363, 0.93293345,
       0.92353372, 0.91303037, 0.90088109, 0.88795046, 0.87600618,
       0.87437662, 0.89264349])


In [ ]:
attr = new_attrs["dod"]["dod_year"]
attr

In [ ]:
h = 5
ds = attr.downsample(column, h)
print(len(np.unique(ds)), np.unique(ds))

us = attr.upsample(ds, h)
print(len(np.unique(us)), np.unique(us))

us = attr.upsample(ds, h, deterministic=False)
print(np.max(us), np.unique(us))

8 [0 1 2 3 4 5 6 7]
8 [ 0  1 19 21 23 26 28 30]
32 [ 0  1  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27
 28 29 30 31 32]


In [ ]:
attr.counts


array([3.91174e+05, 1.72000e+02, 0.00000e+00, 0.00000e+00, 0.00000e+00,
       0.00000e+00, 2.00000e+00, 1.50000e+01, 6.10000e+01, 1.29000e+02,
       1.41000e+02, 1.68000e+02, 1.58000e+02, 2.71000e+02, 4.25000e+02,
       2.63000e+02, 6.68000e+02, 8.65000e+02, 1.10000e+03, 1.43900e+03,
       1.59200e+03, 1.65700e+03, 2.86000e+03, 2.00300e+03, 1.77700e+03,
       1.82300e+03, 1.84400e+03, 1.93600e+03, 1.85600e+03, 1.62800e+03,
       1.67100e+03, 7.48000e+02, 3.51000e+02])


In [ ]:
ZERO_FILL = 1e-24
grouping = new_attrs["dod"]["dod_year"].grouping
counts = new_attrs["dod"]["dod_year"].counts
column = table["dod_year"]

count_list = []
domains = np.max(grouping, axis=1) + 1

for i in range(len(domains)):
    d = domains[i]
    count = np.zeros((d, ))

    for j in range(len(counts)):
        count[grouping[i, j]] += counts[j]

    count_list.append(count)

entropies = []
for count in count_list:
    p = count / count.max() + ZERO_FILL
    entropies.append(-np.sum(p * np.log2(p)))

entropies = np.array(entropies)

entropies, np.log2(domains)


(
    array([0.57051281, 0.57051281, 0.57051281, 0.57051281, 0.57051281,
       0.5704901 , 0.57033927, 0.56983351, 0.56904794, 0.56808503,
       0.56704387, 0.56529822, 0.56361042, 0.56107146, 0.55832771,
       0.55486176, 0.54989038, 0.54361104, 0.53674918, 0.52901496,
       0.52013597, 0.51093399, 0.5012749 , 0.49003703, 0.47908775,
       0.46561534, 0.44977822, 0.43117611, 0.40784666, 0.37856016,
       0.33974806, 0.2700249 ]),
    array([5.043, 5.   , 4.953, 4.906, 4.86 , 4.81 , 4.754, 4.7  , 4.645,
       4.586, 4.523, 4.46 , 4.39 , 4.32 , 4.246, 4.17 , 4.086, 4.   ,
       3.906, 3.807, 3.701, 3.586, 3.459, 3.322, 3.17 , 3.   , 2.807,
       2.586, 2.322, 2.   , 1.585, 1.   ], dtype=float16)
)


In [ ]:
entropies_orig


array([0.57051281, 0.57051281, 0.57051281, 0.57051281, 0.57051281,
       0.5704901 , 0.57033927, 0.56983351, 0.56904794, 0.56808503,
       0.56704387, 0.56529822, 0.56361042, 0.56107146, 0.55832771,
       0.55486176, 0.54989038, 0.54361104, 0.53674918, 0.52901496,
       0.52013597, 0.51093399, 0.5012749 , 0.49003703, 0.47908775,
       0.46561534, 0.44977822, 0.43117611, 0.40784666, 0.37856016,
       0.33974806, 0.2700249 ])


In [ ]:
attr.select_height(1.3)

12
